#Install e Imports

In [ ]:
!pip install soccer-twos

In [ ]:
import torch
import gym
import numpy as np
import os
import soccer_twos
import collections
import random
from collections import deque
from gym_unity.envs import ActionFlattener
from keras import initializers, Input
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Lambda, Flatten
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard
import numpy as np
from tqdm import tqdm

# Definindo o DQN

Tanto o atacante quanto o goleiro.

In [ ]:
class DQNAgentAtacante:
    def __init__(self, input_shape, action_space, **kwargs):
        self.flattener = ActionFlattener(np.array([3, 3, 3], dtype=np.int64))
        self.input_shape = input_shape
        self.action_space = action_space
        self.memory = deque(maxlen=kwargs.get("replay_size", 200000))

        self.gamma = kwargs.get("gamma", 0.9) 
        self.epsilon = kwargs.get("epsilon", 1.0)
        self.epsilon_min = kwargs.get("esilon_min", 0.1)
        self.epsilon_decay = kwargs.get("epsilon_decay", 0.995)
        self.learning_rate = kwargs.get("lr", 1e-3) 
        self.batch_size = kwargs.get("batch_size", 128)

        self.model = self.build_model()
        self.target_network = self.build_model()

    def build_model(self):

        model = Sequential()
        model.add(Input(shape=(336,)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_space.n))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return env.action_space.sample()
        
        else:
            
            #state2 = torch.from_numpy(state).float().unsqueeze(0)
            state2 = np.expand_dims(state,axis=0)
            action_values = self.model.predict(state2)
            action = np.argmax(action_values)
            
            #actions[player_id] = self.flattener.lookup_action(action)
            #actions[player_id] = action

        return action

    def save_in_buffer(self, state, action, reward, next_state, done):
        for i in [0,2]:
            r = 0
            if reward[i] == 0:
                continue
                r = -5
            elif reward[i] == -1:
                r = -1
            elif reward[i] > 0:
                r = reward[i]*1000
            self.memory.append((state[i], action[i], r, next_state[i], done['__all__']))
        #self.memory.append((state, action, reward, next_state, done)) 

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        state, action, reward, next_state, done = zip(*minibatch) # desempacotando as experiências
        state, action, reward, next_state, done = ( 
            np.array(state, dtype='float32'), 
            np.array(action, dtype='int'), 
            np.array(reward, dtype='float32'), 
            np.array(next_state, dtype='float32'), 
            np.array(done, dtype='bool'), )
        
        # Faça aqui a equação que calcula os alvos(targets) para predição ----------------------------------
        # Lembre-se que, para transições terminais (done=True) o alvo/target é apenas a recompensa
        pred = np.max(self.target_network.predict(next_state,verbose=0),axis=1)
        done2 = [int(x) for x in done]
        done2 = np.array(done2)
        targets = reward + (self.gamma*pred*(1-done2))
        # ---------------------------------------------------------------------------------------------------

        target_f = self.target_network.predict(state, verbose=0)
        target_f[np.arange(len(target_f)), action] = targets # aqui alteramos a saída da ação tomada para o alvo

        history = self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
    

In [ ]:
import collections
import random
from collections import deque
from gym_unity.envs import ActionFlattener
from keras import initializers, Input
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Lambda, Flatten
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard
import numpy as np
from tqdm import tqdm

class DQNAgentGoleiro:
    def __init__(self, input_shape, action_space, **kwargs):
        self.flattener = ActionFlattener(np.array([3, 3, 3], dtype=np.int64))
        self.input_shape = input_shape
        self.action_space = action_space
        self.memory = deque(maxlen=kwargs.get("replay_size", 200000))

        self.gamma = kwargs.get("gamma", 0.9) 
        self.epsilon = kwargs.get("epsilon", 1.0)
        self.epsilon_min = kwargs.get("esilon_min", 0.1)
        self.epsilon_decay = kwargs.get("epsilon_decay", 0.995)
        self.learning_rate = kwargs.get("lr", 1e-3) 
        self.batch_size = kwargs.get("batch_size", 128)

        self.model = self.build_model()
        self.target_network = self.build_model()

    def build_model(self):

        model = Sequential()
        model.add(Input(shape=(336,)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_space.n))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return env.action_space.sample()
        
        else:
            
            #state2 = torch.from_numpy(state).float().unsqueeze(0)
            state2 = np.expand_dims(state,axis=0)
            action_values = self.model.predict(state2)
            action = np.argmax(action_values)
            

        return action

    def save_in_buffer(self, state, action, reward, next_state, done):
        for i in [1,3]:
            r = 0
            if reward[i] == 0:
                continue
                r = 1
            elif reward[i] == -1:
                r = -1000
            elif reward[i] > 0:
                r = 3
            self.memory.append((state[i], action[i], r, next_state[i], done['__all__']))
        #self.memory.append((state, action, reward, next_state, done)) 

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        state, action, reward, next_state, done = zip(*minibatch) # desempacotando as experiências
        state, action, reward, next_state, done = ( 
            np.array(state, dtype='float32'), 
            np.array(action, dtype='int'), 
            np.array(reward, dtype='float32'), 
            np.array(next_state, dtype='float32'), 
            np.array(done, dtype='bool'), )
        
        # Faça aqui a equação que calcula os alvos(targets) para predição ----------------------------------
        # Lembre-se que, para transições terminais (done=True) o alvo/target é apenas a recompensa
        pred = np.max(self.target_network.predict(next_state,verbose=0),axis=1)
        done2 = [int(x) for x in done]
        done2 = np.array(done2)
        targets = reward + (self.gamma*pred*(1-done2))
        # ---------------------------------------------------------------------------------------------------

        target_f = self.target_network.predict(state, verbose=0)
        target_f[np.arange(len(target_f)), action] = targets # aqui alteramos a saída da ação tomada para o alvo

        history = self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
    

#Realizando o treinamento

In [ ]:
TEST_EPISODES = 20 # episódios executados para teste
MAX_ITERATIONS = 1000

params = {
    "gamma": 0.9,
    "epsilon": 1.0,
    "esilon_min": 0.1,
    "epsilon_decay": 0.995,
    "lr": 1e-3,
    "replay_size": 500,
    "batch_size": 128,
}
UPDATE_TARGET = 10 # atualizar a target network a cada X passos no ambiente
TEST_EVERY = 200

OBS: a saída abaixa foi interrompida, porém em outras execuções os modelos foram treinados.

In [ ]:
metrics = collections.defaultdict(list)

env = soccer_twos.make(flatten_branched=True)

state_space = env.observation_space
action_space = env.action_space
atacante = DQNAgentAtacante(state_space, action_space, **params)
goleiro = DQNAgentGoleiro(state_space, action_space, **params)
episodes = 1000

for i in range(episodes):
    
    print(f"Episode:{i}")
    done = False

    state = env.reset()

    team0_reward = 0
    team1_reward = 0
    for step in tqdm(range(1000)):
        action = {0:atacante.act(state[0]), 1:goleiro.act(state[1]), 
                  2:env.action_space.sample(), 3:env.action_space.sample()}
        #action = agent.act(state)
        next_state, reward, done, info = env.step(action)

        atacante.save_in_buffer(state, action, reward, next_state, done)
        goleiro.save_in_buffer(state, action, reward, next_state, done)
        step += 1
        team0_reward += reward[0] + reward[1]
        team1_reward += reward[2] + reward[3]
        if done["__all__"]:
            print("Total Reward: ", team0_reward, " x ", team1_reward)
            metrics["team0_reward"].append(team0_reward)
            metrics["team1_reward"].append(team1_reward)
            metrics["episode_len"].append(step)
            break
        else:
            state = next_state

        if step % UPDATE_TARGET == 0:
            atacante.target_network.set_weights(atacante.model.get_weights())
            goleiro.target_network.set_weights(goleiro.model.get_weights())

        if len(atacante.memory) > atacante.batch_size:
            atacante.replay(atacante.batch_size) # a.k.a. learning
        if len(goleiro.memory) > goleiro.batch_size:
            goleiro.replay(goleiro.batch_size) # a.k.a. learning

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0


INFO:mlagents_envs.environment:Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0


[INFO] Connected new brain: SoccerTwos?team=1


INFO:mlagents_envs.environment:Connected new brain: SoccerTwos?team=1


[INFO] Connected new brain: SoccerTwos?team=0


...

1/1 [==============================] - 0s 21ms/step


 25%|██▍       | 246/1000 [01:59<05:55,  2.12it/s]

1/1 [==============================] - 0s 19ms/step


 25%|██▍       | 246/1000 [01:59<06:07,  2.05it/s]


KeyboardInterrupt: ignored

Fechando o ambiente e salvando os modelos:

In [ ]:
env.close()

In [ ]:
atacante.save('/content/drive/MyDrive/atacante.h5')
goleiro.save('/content/drive/MyDrive/goleiro.h5')